# Logistic Regression miRNA
Runs sklearn's [Logistric Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) on miRNA data.  

Three datasets are analyzed using the RandomForest classifier:
* Sham and infeceted mice from all weeks; i.e., 8 week and 16 week datasets are merged.
* Sham and infected mice at 8 weeks.
* Sham and infected mice at 16 weeks.

In each dataset, there is a flag (named ‘infected’) that marks whether the mice came from the infected group or the sham group.  
RandomForestClassifier's variable importance and SHAP values are then used to determine which miRNA variable was most important in each cohort dataset.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# trying to supress sklearn warning
import os
os.environ["PYTHONWARNINGS"] = "ignore::UserWarning"
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import \
    RandomizedSearchCV, GridSearchCV, RepeatedKFold, cross_val_score, train_test_split
from sklearn.metrics import \
    accuracy_score, f1_score, recall_score, classification_report, \
    roc_auc_score, roc_curve, mean_absolute_error
from IPython.display import display
import random
import shap

from util import \
    make_mirna_nanostring_df, make_study_df, param_search_cv, \
    param_search, plot_shap_feature_importance, plot_xgb_feature_importance, \
    combine_shap_heatmaps, plot_shap_summary, plot_shap_heatmap, \
    plot_shap_dependence, plot_shap_importance_with_summary, top_shap_values

In [4]:
! python --version

Python 3.13.0


In [5]:
print('sklearn version', sklearn.__version__)
print('shap version', shap.__version__)

sklearn version 1.3.0
shap version 0.42.1


---

## load miRNA data

In [6]:
file_names = [
    '../data/Fusobacterium_nucleatum/8 weeks F. nucleatum infection.csv',
    '../data/Fusobacterium_nucleatum/8 weeks SHAM infection.csv',
    '../data/Fusobacterium_nucleatum/16 weeks F. nucleatum infection.csv',
    '../data/Fusobacterium_nucleatum/16 weeks SHAM infection.csv',
    '../data/Porphyromonas_gingivalis/8 weeks P. gingivalis NanoString Data.csv',
    '../data/Porphyromonas_gingivalis/8 weeks SHAM RAW data.csv',
    '../data/Porphyromonas_gingivalis/16 weeks P. gingivalis  NanoString Data.csv',
    '../data/Porphyromonas_gingivalis/16 weeks SHAM RAW data.csv',
    '../data/Streptococcus_gordonii/8 weeks S.gordonii infection.csv',
    '../data/Streptococcus_gordonii/8 weeks SHAM infection.csv',
    '../data/Streptococcus_gordonii/16 weeks S. gordonii infection.csv',
    '../data/Streptococcus_gordonii/16 weeks SHAM infection.csv',
    '../data/Tannerella_forsythia/8 weeks T. forsythia bacteria infected mice RAW data.csv',
    '../data/Tannerella_forsythia/8 weeks SHAM RAW data.csv',
    '../data/Tannerella_forsythia/16 weeks T. forsythia bacteria infected mice RAW data.csv',
    '../data/Tannerella_forsythia/16 weeks SHAM RAW data.csv',
    '../data/Treponema_denticola/8 weeks T. denticola bacteria infected mice RAW data.csv',
    '../data/Treponema_denticola/8 weeks SHAM RAW data.csv',
    '../data/Treponema_denticola/16 weeks T. denticola bacteria infected mice RAW data.csv',
    '../data/Treponema_denticola/16 weeks SHAM RAW data.csv'
]
cohort_names = [
    'inf_fn_8_weeks', 'sham_fn_8_weeks', 'inf_fn_16_weeks', 'sham_fn_16_weeks',
    'inf_pg_8_weeks', 'sham_pg_8_weeks', 'inf_pg_16_weeks', 'sham_pg_16_weeks',
    'inf_sg_8_weeks', 'sham_sg_8_weeks', 'inf_sg_16_weeks', 'sham_sg_16_weeks',
    'inf_tf_8_weeks', 'sham_tf_8_weeks', 'inf_tf_16_weeks', 'sham_tf_16_weeks',
    'inf_td_8_weeks', 'sham_td_8_weeks', 'inf_td_16_weeks', 'sham_td_16_weeks'
]
list(zip(file_names, cohort_names))

[('../data/Fusobacterium_nucleatum/8 weeks F. nucleatum infection.csv',
  'inf_fn_8_weeks'),
 ('../data/Fusobacterium_nucleatum/8 weeks SHAM infection.csv',
  'sham_fn_8_weeks'),
 ('../data/Fusobacterium_nucleatum/16 weeks F. nucleatum infection.csv',
  'inf_fn_16_weeks'),
 ('../data/Fusobacterium_nucleatum/16 weeks SHAM infection.csv',
  'sham_fn_16_weeks'),
 ('../data/Porphyromonas_gingivalis/8 weeks P. gingivalis NanoString Data.csv',
  'inf_pg_8_weeks'),
 ('../data/Porphyromonas_gingivalis/8 weeks SHAM RAW data.csv',
  'sham_pg_8_weeks'),
 ('../data/Porphyromonas_gingivalis/16 weeks P. gingivalis  NanoString Data.csv',
  'inf_pg_16_weeks'),
 ('../data/Porphyromonas_gingivalis/16 weeks SHAM RAW data.csv',
  'sham_pg_16_weeks'),
 ('../data/Streptococcus_gordonii/8 weeks S.gordonii infection.csv',
  'inf_sg_8_weeks'),
 ('../data/Streptococcus_gordonii/8 weeks SHAM infection.csv',
  'sham_sg_8_weeks'),
 ('../data/Streptococcus_gordonii/16 weeks S. gordonii infection.csv',
  'inf_sg_16_

In [7]:
miRNA_df = make_mirna_nanostring_df(file_names, cohort_names)
miRNA_df.shape

(200, 604)

## create dataframes and X, y datasets

In [8]:
df_8_weeks = make_study_df(miRNA_df, cohort_str='8_weeks', infected_str='inf_')
df_16_weeks = make_study_df(miRNA_df, cohort_str='16_weeks', infected_str='inf_')
df_all_weeks = make_study_df(miRNA_df, infected_str='inf_')

In [9]:
X_8_weeks, y_8_weeks = df_8_weeks.drop('infected', axis=1), df_8_weeks['infected']
X_16_weeks, y_16_weeks = df_16_weeks.drop('infected', axis=1), df_16_weeks['infected']
X_all_weeks, y_all_weeks = df_all_weeks.drop('infected', axis=1), df_all_weeks['infected']

---

## search for best params

In [10]:
param_grid = {
    'penalty' : ['l1','l2'], 
    'C': [0.1, 1, 10, 100, 1000], 
    'solver'  : ['newton-cg', 'lbfgs', 'liblinear'],
    'max_iter': [50, 100, 150, 200]
}

In [11]:
# used for testing single param search
# params_8_weeks = \
#     param_search(param_grid, X_8_weeks, y_8_weeks, LogisticRegression, cv=5, search_type='grid')

### Run GridSearchCV to find best params.

In [12]:
cv_range = [3, 5, 10]
clf = LogisticRegression

print('** 8 weeks params **')
params_8_weeks_cv = \
    param_search_cv(
        param_grid, X_8_weeks, y_8_weeks, clf, cvs=cv_range, search_type='grid', print_best=True)

print('** 16 weeks params **')
params_16_weeks_cv = \
    param_search_cv(
        param_grid, X_16_weeks, y_16_weeks, clf, cvs=cv_range, search_type='grid', print_best=True)

print('** All weeks params **')
params_8_weeks_cv = \
    param_search_cv(
        param_grid, X_all_weeks, y_all_weeks, clf, cvs=cv_range, search_type='grid', print_best=True)

** 8 weeks params **
{'cv': 10,
 'params': {'C': 1000, 'max_iter': 200, 'penalty': 'l1', 'solver': 'liblinear'},
 'score': 0.97}
** 16 weeks params **
{'cv': 10,
 'params': {'C': 100, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'},
 'score': 0.99}
** All weeks params **


/Users/duncanw/repos/uflcod/miRNA-periodontal-disease/venv/lib/python3.11/site-packages/scipy/optimize/_linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/duncanw/repos/uflcod/miRNA-periodontal-disease/venv/lib/python3.11/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/duncanw/repos/uflcod/miRNA-periodontal-disease/venv/lib/python3.11/site-packages/scipy/optimize/_linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/duncanw/repos/uflcod/miRNA-periodontal-disease/venv/lib/python3.11/site-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not co

{'cv': 5,
 'params': {'C': 1000, 'max_iter': 150, 'penalty': 'l1', 'solver': 'liblinear'},
 'score': 0.97}


### assign best params

In [13]:
params_8_weeks = {'C': 0.1, 'max_iter': 50, 'penalty': 'l2', 'solver': 'newton-cg'}
 
params_16_weeks = {'C': 10, 'max_iter': 50, 'penalty': 'l2', 'solver': 'newton-cg'}

params_all_weeks = {'C': 10, 'max_iter': 100, 'penalty': 'l2', 'solver': 'lbfgs'}

---

### compute Logistic Regression models

In [14]:
model_8_weeks = LogisticRegression(**params_8_weeks).fit(X_8_weeks, y_8_weeks)
model_16_weeks = LogisticRegression(**params_16_weeks).fit(X_16_weeks, y_16_weeks)
model_all_weeks = LogisticRegression(**params_all_weeks).fit(X_all_weeks, y_all_weeks)

In [15]:
print ('model_8_weeks', 'f1 score:', f1_score(y_8_weeks, model_8_weeks.predict(X_8_weeks)))
print ('model_16_weeks', 'f1 score:', f1_score(y_16_weeks, model_16_weeks.predict(X_16_weeks)))
print ('model_all_weeks', 'f1 score:', f1_score(y_all_weeks, model_all_weeks.predict(X_all_weeks)))

model_8_weeks f1 score: 1.0
model_16_weeks f1 score: 1.0
model_all_weeks f1 score: 0.9847715736040609


---

### compute SHAP values (used for explainablity)

In [16]:
# need to change int and bool to classes int_ and bool_ to handle error 'numpy' has no attribute 'int'/'bool'/etc.
np.bool = np.bool_
np.int = np.int_
np.float = np.float_

In [21]:
shap.sample(X_8_weeks, nsamples=20, random_state=1989)

,miR-200b,miR-300,miR-125a-3p,miR-16,miR-669m,miR-192,miR-1907,miR-290-3p,miR-682,miR-491,...,miR-464,miR-669a,miR-453,miR-1932,miR-M1-1,miR-1894-3p,miR-715,miR-M1-8,miR-201,miR-466i
8,1182,177,152,12288,25,113,19,120,122,84,...,103,157,10,17,93,22,23,145,78,22
1,279,108,109,4899,15,90,14,104,84,86,...,73,100,16,11,68,16,20,102,51,26
0,451,138,133,17007,37,119,25,126,100,86,...,81,123,15,22,100,25,29,120,87,23
9,311,144,158,8620,20,136,31,126,128,109,...,109,139,22,20,103,33,26,155,87,23
4,191,85,92,7181,19,62,11,78,61,41,...,52,76,13,18,48,24,10,87,40,27
9,381,174,172,11203,31,175,22,181,153,122,...,136,163,25,12,140,22,26,188,142,21
2,205,121,105,13658,35,81,13,90,60,53,...,58,120,28,9,75,36,30,82,50,29
8,1182,177,152,12288,25,113,19,120,122,84,...,103,157,10,17,93,22,23,145,78,22
5,351,126,115,16355,39,88,16,116,81,71,...,61,131,14,18,69,27,27,96,60,27
4,210,131,111,4964,22,110,17,107,92,75,...,86,138,13,10,78,15,18,138,63,19


In [24]:
X_8_weeks.shape

(100, 600)

In [23]:
explainer_8_weeks = shap.KernelExplainer(
    model_8_weeks.predict, shap.sample(X_8_weeks, nsamples=25, random_state=1989), feature_names=X_8_weeks.columns.tolist()
)
shap_values_8_weeks = explainer_8_weeks(X_8_weeks)

# explainer_16_weeks = shap.KernelExplainer(
#     model_16_weeks.predict, X_16_weeks, feature_names=X_16_weeks.columns.tolist()
# )
# shap_values_16_weeks = explainer_16_weeks(X_16_weeks)

# explainer_all_weeks = shap.KernelExplainer(
#     model_all_weeks.predict, X_all_weeks, feature_names=X_all_weeks.columns.tolist()
# )
# shap_values_all_weeks = explainer_all_weeks(X_all_weeks)

  0%|          | 0/100 [00:00<?, ?it/s]

---

## Bar and summary plots of feature importance

In [ ]:
plot_shap_importance_with_summary(
    shap_values_8_weeks, 
    X_8_weeks, 
    title='',
    save_fig=True,
    file_name='all_pathogens_lr_SHAP_feature_importance_with_summary_8_weeks.png'
)

plot_shap_importance_with_summary(
    shap_values_16_weeks, 
    X_16_weeks, 
    title='',
    save_fig=True,
    file_name='all_pathogens_lr_SHAP_feature_importance_with_summary_16_weeks.png'
)

plot_shap_importance_with_summary(
    shap_values_all_weeks, 
    X_all_weeks, 
    title='',
    supxlabel = 'SHAP value (impact on model output)',
    save_fig=True,
    file_name='all_pathogens_lr_SHAP_feature_importance_with_summary_all_weeks.png'
)

## SHAP heatmaps

In [ ]:
plot_shap_heatmap(
    top_shap_values(shap_values_8_weeks), 
    "SHAP Heatmap 8 Weeks\nLogistic Regression", 
    x_label='Instances',
    save_fig=True,
    file_name='all_pathogens_lr_SHAP_Heatmap_8_Weeks.png'
)

plot_shap_heatmap(
    top_shap_values(shap_values_16_weeks), 
    "SHAP Heatmap 16 Weeks\nLogistic Regression", 
    x_label='Instances',
    save_fig=True,
    file_name='all_pathogens_lr_SHAP_Heatmap_16_Weeks.png'
)

plot_shap_heatmap(
    top_shap_values(shap_values_all_weeks), 
    "SHAP Heatmap All Weeks\nLogistic Regression", 
    x_label='Instances',
    save_fig=True,
    file_name='all_pathogens_lr_SHAP_Heatmap_All_Weeks.png'
)

## SHAP dependence plots

In [ ]:
plot_shap_dependence(
    shap_values_8_weeks, 
    X_8_weeks, 
    "SHAP Dependence Plot 8 Weeks\nLogistic Regression", 
    save_fig=True,
    file_name='all_pathogens_lr_SHAP_Dependence_Plot_8_Weeks.png'
)

plot_shap_dependence(
    shap_values_16_weeks, 
    X_16_weeks, 
    "SHAP Dependence Plot 16 Weeks\nLogistic Regression", 
    save_fig=True,
    file_name='all_pathogens_lr_SHAP_Dependence_Plot_16_Weeks.png'
)

plot_shap_dependence(
    shap_values_all_weeks, 
    X_all_weeks, 
    "SHAP Dependence Plot All Weeks\nLogistic Regression", 
    save_fig=True,
    file_name='all_pathogens_lr_SHAP_Dependence_Plot_All_Weeks.png'
)